# DATA ANALYSIS PROJECT

## INDIAN START-UPS FUNDING. INSIGHTS AND TRENDS FROM 2018 TO 2021

### Project Description
This data analysis project focuses on the funding received by start-ups in India from 2018 to 2021. <br><br>The objective is to gain insights into the ecosystem and propose the best course of action for our team's venture. <br><br>By analyzing the data on funding amounts, start-up details, and investor information, we aim to unearth prevailing patterns and gain insights about the opportunities in India's start-up ecosystem to inform decision-making.

### Taking Preview of the Data at Hand

Data from four different sources were gathered for this project. Two from Microsoft SQL Server, one from onedrive and the last one from a github repo.

We set the van off with loading the needed libraries.

In [1568]:
# Load some libraries

import pyodbc
from dotenv import dotenv_values
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.impute import SimpleImputer
import re
import warnings

warnings.filterwarnings('ignore')

# Display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Convert amounts from scientific format
pd.set_option('display.float_format', '{:.1f}'.format)

In [1569]:
# Get the values for the database credentials set in the '.env' file
env_var = {key.upper(): value for key, value in dotenv_values('.env').items()}

# Unpack the values from the dictionary
database, username, password, server = (env_var.get(key) for key in ['DATABASE', 'USERNAME', 'PASSWORD', 'SERVER'])

connection_string = f"DRIVER={{SQL Server}}; SERVER={server};DATABASE={database};UID={username};PWD={password}"

In [1570]:
# Connect to database

connection = pyodbc.connect(connection_string)

Success! Now, we read the data using sql SELECT statement and pandas' read_sql.

In [1571]:
# Loading the data for 2020

query2020 = "select * from dbo.LP1_startup_funding2020"

data2020 = pd.read_sql(query2020, connection)

In [1572]:
# Loading the data for 2021

query2021 = "select * from dbo.LP1_startup_funding2021"

data2021 = pd.read_sql(query2021, connection)

In [1573]:
# Close the database connection for scalability and resource management.

connection.close()

Perfect! We hop onto the other datasets obtained in '.csv' format

In [1574]:
# Loading data for 2018 & 2019


data2018 = pd.read_csv('startup_funding2018.csv')
data2019 = pd.read_csv('startup_funding2019.csv')

### Data Inspection

### 2021 Data

In [1575]:
data2021.head(10) # View first 10 rows of data2021

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First","$1,200,000",Pre-series A
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management","$120,000,000",None
2,Lead School,2012.0,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital","$30,000,000",Series D
3,Bizongo,2015.0,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital","$51,000,000",Series C
4,FypMoney,2021.0,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal","$2,000,000",Seed
5,Urban Company,2014.0,New Delhi,Home services,Urban Company (Formerly UrbanClap) is a home a...,"Abhiraj Singh Bhal, Raghav Chandra, Varun Khaitan",Vy Capital,"$188,000,000",None
6,Comofi Medtech,2018.0,Bangalore,HealthTech,Comofi Medtech is a healthcare robotics startup.,Gururaj KB,"CIIE.CO, KIIT-TBI","$200,000",None
7,Qube Health,2016.0,Mumbai,HealthTech,India's Most Respected Workplace Healthcare Ma...,Gagan Kapur,Inflection Point Ventures,Undisclosed,Pre-series A
8,Vitra.ai,2020.0,Bangalore,Tech Startup,Vitra.ai is an AI-based video translation plat...,Akash Nidhi PS,Inflexor Ventures,Undisclosed,None
9,Taikee,2010.0,Mumbai,E-commerce,"Taikee is the ISO-certified, B2B e-commerce pl...","Nidhi Ramachandran, Sachin Chhabra",None,"$1,000,000",None


In [1576]:
# Get summarised info about missing values and data types of the columns in data2021

data2021.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1209 entries, 0 to 1208
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Company_Brand  1209 non-null   object 
 1   Founded        1208 non-null   float64
 2   HeadQuarter    1208 non-null   object 
 3   Sector         1209 non-null   object 
 4   What_it_does   1209 non-null   object 
 5   Founders       1205 non-null   object 
 6   Investor       1147 non-null   object 
 7   Amount         1206 non-null   object 
 8   Stage          781 non-null    object 
dtypes: float64(1), object(8)
memory usage: 85.1+ KB


From the displayed information about data2021,:

* *Founded*, *HeadQuarter*, *Founders*, *Investor*, *Amount* and *Stage* columns have some missing data
* All columns are of the object data type except *Founded* which is a float.

We take a critical look at all the data

In [1577]:
pd.set_option('display.max_rows', None) # Set option for the full display of all rows


data2021

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019.0,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First","$1,200,000",Pre-series A
1,upGrad,2015.0,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management","$120,000,000",None
2,Lead School,2012.0,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital","$30,000,000",Series D
3,Bizongo,2015.0,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital","$51,000,000",Series C
4,FypMoney,2021.0,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal","$2,000,000",Seed
5,Urban Company,2014.0,New Delhi,Home services,Urban Company (Formerly UrbanClap) is a home a...,"Abhiraj Singh Bhal, Raghav Chandra, Varun Khaitan",Vy Capital,"$188,000,000",None
6,Comofi Medtech,2018.0,Bangalore,HealthTech,Comofi Medtech is a healthcare robotics startup.,Gururaj KB,"CIIE.CO, KIIT-TBI","$200,000",None
7,Qube Health,2016.0,Mumbai,HealthTech,India's Most Respected Workplace Healthcare Ma...,Gagan Kapur,Inflection Point Ventures,Undisclosed,Pre-series A
8,Vitra.ai,2020.0,Bangalore,Tech Startup,Vitra.ai is an AI-based video translation plat...,Akash Nidhi PS,Inflexor Ventures,Undisclosed,None
9,Taikee,2010.0,Mumbai,E-commerce,"Taikee is the ISO-certified, B2B e-commerce pl...","Nidhi Ramachandran, Sachin Chhabra",None,"$1,000,000",None


### Issues arising from data2021:

* Founded column is a float. It has to be a date

* Some Amounts have $undisclosed, Undisclosed and undisclosed. We may treat them as missing values

* The Amounts and Stage for FanPlay company at index 98 are interchanged

* At index 242, 256, 257, and 545, the amount appears at the investor’s column and then stage at the amount column

* Little Leap at   at index 538 has Ah! Ventures(investors) instead of amount and then amount at stage column. Also, ‘Holistic Development Programs for children in …; should be   replaced with Vishal Gupta as founder.

* BHyve company index 551 has part of ‘what_it_does’ at founders cl then founders at investor cl and investors at amount col ……. It also had investor’s name as ITO Angel Network instead of JITO Angel Network

* Some amounts are separated by ‘,’ some too have ‘$$’ preceding them and some only ‘$’ as amount…this is for EventBeep, MPL

* Amount for Godamwale at index 1148 is misspelt as 1000000\t#REF! instead of 1000000 and is at investor col with stage rather taking its place. Also, the investor is **Capt. Anand Aryamane**

* for index 1100-Sochcast company, Heaquarter is replaced with ‘Online Media\t#REF!’ and sector
* There are some duplicates

### Resolution

* Delete duplicates
* Update misplaced values for the specifi rows and columns
* Impute with right values from credible sources where such values are mispelt or omitted
* Convert columns to right data types
* Standardise the 'Stage' column for good analysis by renaming or grouping

#### Delete Duplicates

In [1578]:
# Confirm duplicated records

data2021.duplicated().sum()

19

Let's take a look, first.

In [1579]:
data2021[data2021.duplicated()].head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
107,Curefoods,2020.0,Bangalore,Food & Beverages,Healthy & nutritious foods and cold pressed ju...,Ankit Nagori,"Iron Pillar, Nordstar, Binny Bansal",$13000000,None
109,Bewakoof,2012.0,Mumbai,Apparel & Fashion,Bewakoof is a lifestyle fashion brand that mak...,Prabhkiran Singh,InvestCorp,$8000000,None
111,FanPlay,2020.0,Computer Games,Computer Games,A real money game app specializing in trivia g...,YC W21,"Pritesh Kumar, Bharat Gupta",Upsparks,$1200000
117,Advantage Club,2014.0,Mumbai,HRTech,Advantage Club is India's largest employee eng...,"Sourabh Deorah, Smiti Bhatt Deorah","Y Combinator, Broom Ventures, Kunal Shah",$1700000,None
119,Ruptok,2020.0,New Delhi,FinTech,Ruptok fintech Pvt. Ltd. is an online gold loa...,Ankur Gupta,Eclear Leasing,$1000000,None


Delete them

In [1580]:
# Delete the duplicates and set the index

data2021 = data2021.drop_duplicates().reset_index(drop=True)

In [1581]:
# Check if duplicates were dropped

data2021.shape

(1190, 9)

Swap values interchanged

In [1582]:
# A function to swap vulues un specif columns

def swap_values(df, row_index, column1, column2):
    """
    Swaps the values of two specific columns in a DataFrame for a given row index.

    Args:
        df (pd.DataFrame): The DataFrame to be modified.
        row_index: The index of the row where the values will be swapped.
        column1 (str): The name of the first column.
        column2 (str): The name of the second column.

    Returns:
        pd.DataFrame: The DataFrame with the swapped values.
    """
    df_modified = df.copy()
    value1 = df_modified.at[row_index, column1]
    value2 = df_modified.at[row_index, column2]
    df_modified.at[row_index, column1] = value2
    df_modified.at[row_index, column2] = value1
    return df_modified


In [1583]:
# Apply function to swap Amount and Stage cols

data2021 = swap_values(data2021, 98 , 'Amount', 'Stage')
data2021 = swap_values(data2021, 655 , 'Amount', 'Stage')

#### Update misplaced values

We now update values wrongly placed or mispelt

In [1584]:
# A function to update values in specific columns and rows

def update_values(df, row_index, column_values):
    """
    Updates multiple values in a DataFrame for a specific row.

    Args:
        df (pd.DataFrame): The DataFrame to be modified.
        row_index: The index of the row where the values will be updated.
        column_values (dict): Dictionary where keys are column names and values are the new values.

    Returns:
        pd.DataFrame: The DataFrame with the updated values.
    """
    df_modified = df.copy()
    for column, value in column_values.items():
        df_modified.at[row_index, column] = value
    return df_modified



In [1585]:
# Apply function to specific rows and columns

data2021 = update_values(data2021, 237, column_values={'Investor': 'Unknown', 'Amount': '$22000000', 'Stage': 'Series C'})

data2021 = update_values(data2021, 238, column_values={'HeadQuarter':'Gurugram', 'Investor': 'Unknown', 'Amount': '$5000000', 'Stage': 'Seed'})

data2021 = update_values(data2021, 526, column_values={'Investor': 'Unknown', 'Amount': '$1000000', 'Stage': 'Pre-series A'})

data2021 = update_values(data2021, 532, column_values={'Founders': 'Omkar Pandharkame, Ketaki Ogale' ,'Investor': 'JITO Angel Network, LetsVenture', 'Amount': '$300000', 'Stage': 'Venture-Series Unknown'})

data2021 = update_values(data2021, 658, column_values={'Founders': 'Sushil Agarwal' ,'Investor': 'JITO Angel Network, LetsVenture', 'Amount': '$1000000', 'Stage': 'Venture-Series Unknown'})

# data2021 = update_values(data2021, 1100, column_values={'HeadQuarter': 'Bangalore' ,'Investor': ' VINNERS, Raj Nayak, Viacom18 Media, Amritaanshu Agrawal, Mirabaud', 'Amount': '$Undisclosed', 'Stage': 'Venture-Series Unknown'})

data2021 = update_values(data2021, 519, column_values={'Founders': 'Vishal Gupta' ,'Investor': 'ah! Ventures', 'Amount': '$300000', 'Stage': 'Venture-Series Unknown'})

data2021 = update_values(data2021, 1148, column_values={ 'Investor': 'Capt. Anand Aryamane', 'Amount': '$1000000', 'Stage': 'Seed'})


data2021 = update_values(data2021, 1129, column_values={ 'Investor': 'Capt. Anand Aryamane', 'Amount': '$1000000', 'Stage': 'Seed'})

In [1586]:
# Check just by one example

data2021.iloc[[1148]]

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
1148,Coutloot,2016.0,Mumbai,Consumer Services,Empowering local markets to sell online social...,"Mahima Kaul, Jasmeet Thind",Capt. Anand Aryamane,$1000000,Seed


### Company_Brand column

In [1587]:
# Get unique values with their frequencies of the comapany_brand column

data2021['Company_Brand'].value_counts()


BharatPe                           8
BYJU'S                             4
Teachmint                          3
OYO                                3
Captain Fresh                      3
Jumbotail                          3
Karbon Card                        3
MediBuddy                          3
Ola Electric                       3
Delhivery                          3
Zetwerk                            3
KreditBee                          3
Siply                              3
CoinSwitch Kuber                   3
Spinny                             3
Northern Arc                       3
Slice                              3
DealShare                          3
MPL                                3
Bizongo                            3
Camp K12                           2
Dream Sports                       2
Bewakoof                           2
Chalo                              2
Instoried                          2
SATYA MicroCapital                 2
DeHaat                             2
T

Same company name was treated as separate identities: BYJU’S and  BYJU'S. <br>This might be due to the different apostrophies used.

........let's check!

In [1588]:
# Check if they are same

"BYJU'S" ==  'BYJU’S'


False

In [1589]:
# Good! Now we correct that and that's all for company_Brand

data2021['Company_Brand'] = data2021['Company_Brand'].str.replace("’", "'")

In [1590]:
data2021['Company_Brand'].nunique()

1032

data2021['Company_Brand'].nunique()

In [1591]:
# Get summary info, number of unique values and the values of the 'HeadQuarter' column

print(data2021['HeadQuarter'].info())
print(data2021['HeadQuarter'].nunique())
print(data2021['HeadQuarter'].value_counts())

<class 'pandas.core.series.Series'>
RangeIndex: 1190 entries, 0 to 1189
Series name: HeadQuarter
Non-Null Count  Dtype 
--------------  ----- 
1189 non-null   object
dtypes: object(1)
memory usage: 9.4+ KB
None
69
Bangalore                            421
Mumbai                               198
Gurugram                             155
New Delhi                            142
Pune                                  46
Chennai                               40
Noida                                 34
Hyderabad                             32
Jaipur                                11
Ahmedabad                             11
Thane                                  7
Gurgaon                                7
Ahmadabad                              6
Kolkata                                5
Chandigarh                             5
Haryana                                4
Gujarat                                3
Lucknow                                3
Coimbatore                             3
Ghaziab

There sure have been wrong placement for 'Information Technology & Services' and  'Food & Beverages' <br>
Let's check that out!

In [1592]:
# Checking .....

data2021[data2021['HeadQuarter'] == 'Food & Beverages']

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
236,MasterChow,2020.0,Food & Beverages,Hauz Khas,A ready-to-cook Asian cuisine brand,"Vidur Kataria, Sidhanth Madan",WEH Ventures,$461000,Seed


In [1593]:
# Placing them correctly for MasterChow using the update_value function

data2021 = update_values(data2021, 236, column_values={'HeadQuarter': 'New Delhi', 'Sector': 'Food & Beverages'})

In [1594]:
# Checking .....

data2021[data2021['HeadQuarter'] == 'Food & Beverages']

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage


In [1595]:
# Checking for Peak

data2021[data2021['HeadQuarter'] == 'Information Technology & Services']

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
1157,Peak,2014.0,Information Technology & Services,"Manchester, Greater Manchester",Peak helps the world's smartest companies put ...,Atul Sharma,SoftBank Vision Fund 2,$75000000,Series C


In [1596]:
# Placing them correctly

data2021 = update_values(data2021, 1157, column_values={ 'HeadQuarter': 'Manchestor', 'Sector': 'Information Technology & Services'})

In [1597]:
# Checking ......

data2021[data2021['HeadQuarter'] == 'Information Technology & Services']


,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage


Deal with Missing Values

In [1598]:
# Check for missing values

data2021[data2021['HeadQuarter'].isnull()]

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
29,Vidyakul,2018.0,None,EdTech,Vidyakul is an vernacular e-learning platform ...,"Raman Garg, Tarun Saini","JITO Angel Network, SOSV","$500,000",Seed


In [1599]:
# Fill the missing value with its correct value 'Gurgaon'ArithmeticError
data2021 = update_values(data2021, 29, column_values={'HeadQuarter': 'Gurgaon'})

#### Dealing with the 'Founded' Col

In [1600]:
# Getting the unique values of the 'Founded' column

data2021['Founded'].unique()

array([2019., 2015., 2012., 2021., 2014., 2018., 2016., 2020., 2010.,
       2017., 1993., 2008., 2013., 1999., 1989., 2011.,   nan, 2009.,
       2002., 1994., 2006., 2000., 2007., 1978., 2003., 1998., 1991.,
       1984., 2004., 2005., 1963.])

In [1601]:
# Checking the number of missing values

data2021['Founded'].isnull().sum()

1

In [1602]:
# Get statistical summaries of the column to inform 
# what strategy to use for imputation

data2021['Founded'].describe()

count   1189.0
mean    2016.6
std        4.5
min     1963.0
25%     2015.0
50%     2018.0
75%     2020.0
max     2021.0
Name: Founded, dtype: float64

In [1603]:
# Check data type 


data2021['Founded'].dtype

dtype('float64')

Now, we can do imputation. <br> Since 'year' is a categorical data here, we use the most frequent to fill the null.

In [1604]:
# Imputing missing values

array = data2021['Founded'].values.reshape(-1,1)
imputer = SimpleImputer(strategy='most_frequent')

data2021['Founded'] = imputer.fit_transform(array)

In [1605]:
data2021['Founded'].unique()

array([2019., 2015., 2012., 2021., 2014., 2018., 2016., 2020., 2010.,
       2017., 1993., 2008., 2013., 1999., 1989., 2011., 2009., 2002.,
       1994., 2006., 2000., 2007., 1978., 2003., 1998., 1991., 1984.,
       2004., 2005., 1963.])

Converting 'Founded' from float to int64

In [1606]:
# Founded column converted to int

data2021['Founded'] = data2021['Founded'].astype(int)
data2021['Founded'].head()

0    2019
1    2015
2    2012
3    2015
4    2021
Name: Founded, dtype: int32

In [1607]:
# Making sure the conversion was done

data2021['Founded'].dtype

dtype('int32')

In [1608]:
# Get unique values

data2021['Founded'].unique()

array([2019, 2015, 2012, 2021, 2014, 2018, 2016, 2020, 2010, 2017, 1993,
       2008, 2013, 1999, 1989, 2011, 2009, 2002, 1994, 2006, 2000, 2007,
       1978, 2003, 1998, 1991, 1984, 2004, 2005, 1963])

### Cleaning the 'Amount' col

In [1609]:
# Inspecting unique values with their frequencies

data2021['Amount'].value_counts()

$Undisclosed        72
$1000000            54
Undisclosed         43
$2000000            40
$1,000,000          33
$3000000            31
$5000000            30
$10000000           28
$500000             24
$undisclosed        22
$4000000            20
$2,000,000          20
$300000             19
$200000             19
$400000             18
$3,000,000          17
$200,000            17
$20000000           15
$10,000,000         15
$6000000            14
$1200000            13
$7000000            13
$30000000           13
$5,000,000          12
$15,000,000         11
$100000000          11
$700000             11
$500,000            11
$4,000,000          10
$300,000            10
$600000             10
$6,000,000          10
$50000000           10
$1500000            10
$1,500,000           9
$12000000            9
$30,000,000          8
$100000              8
$2500000             8
$40000000            8
$3500000             7
$8000000             7
$25000000            6
$4500000   

Replacing Undisclosed/$undisclosed/$Undisclosed values under the 'Amounts' Column

In [1610]:
# Function to replace anomalous amount values with np.nan

def replace_values_with_nan(df, column_name, values_to_replace):
    """
    Replaces specified values in a column of a DataFrame with np.nan.

    Args:
        df (pandas.DataFrame): The DataFrame containing the column to modify.
        column_name (str): The name of the column to replace values in.
        values_to_replace (list or scalar): The value(s) to replace with np.nan. Can be a single value or a list of values.

    Returns:
        pandas.DataFrame: A modified DataFrame with the specified values replaced by np.nan.

    Example:
        # Create a sample DataFrame
        data = {
            'Column1': [1, 2, 3, 4, 5],
            'Column2': ['A', 'B', 'C', 'D', 'E'],
            'Column3': ['X', 'Y', 'Z', 'X', 'Z']
        }

        df = pd.DataFrame(data)

        # Define the column name and values to replace with np.nan
        column_name = 'Column3'
        values_to_replace = ['X', 'Z']

        # Call the replace_values_with_nan function
        df_modified = replace_values_with_nan(df, column_name, values_to_replace)

        # Print the modified DataFrame
        print(df_modified)
    """
    df[column_name] = df[column_name].replace(values_to_replace, np.nan)
    return df


In [1611]:
# Invoking the function to replace the missing values with np.nan

data2021 = replace_values_with_nan(data2021, 'Amount', ['$Undisclosed', '$undisclosed', 'undisclosed', 'Undisclosed', 'None'])

# Replacing the undesired characters

data2021['Amount'] = data2021['Amount'].str.replace('[$,]', '')

In [1612]:
# Converting the 'Amount' col to float

data2021['Amount'] = pd.to_numeric(data2021['Amount'])

In [1613]:
# Checking....

data2021['Amount'].dtype

dtype('float64')

In [1614]:
# Make scientific numbers display as normal with i decimal point

pd.set_option('display.float_format', '{:.1f}'.format)


In [1615]:
# Get statistical summary of data2021

data2021.describe()

,Founded,Amount
count,1190.0,1049.0
mean,2016.6,171212131.6
std,4.5,4631144962.5
min,1963.0,10000.0
25%,2015.0,1000000.0
50%,2018.0,3500000.0
75%,2020.0,15000000.0
max,2021.0,150000000000.0


Filling missing values in the 'Amount' Col

In [1616]:
# Imputing missing values

array = data2021['Amount'].values.reshape(-1,1) # simpleImputer takes 2d array as input
imputer = SimpleImputer(strategy='median')   # Chose 'median' strategy because of the presence of outliers

data2021['Amount'] = imputer.fit_transform(array)

In [1617]:
# View first 5 rows of data2021

data2021.head()

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
0,Unbox Robotics,2019,Bangalore,AI startup,Unbox Robotics builds on-demand AI-driven ware...,"Pramod Ghadge, Shahid Memon","BEENEXT, Entrepreneur First",1200000.0,Pre-series A
1,upGrad,2015,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management",120000000.0,None
2,Lead School,2012,Mumbai,EdTech,LEAD School offers technology based school tra...,"Smita Deorah, Sumeet Mehta","GSV Ventures, Westbridge Capital",30000000.0,Series D
3,Bizongo,2015,Mumbai,B2B E-commerce,Bizongo is a business-to-business online marke...,"Aniket Deb, Ankit Tomar, Sachin Agrawal","CDC Group, IDG Capital",51000000.0,Series C
4,FypMoney,2021,Gurugram,FinTech,"FypMoney is Digital NEO Bank for Teenagers, em...",Kapil Banwari,"Liberatha Kallat, Mukesh Yadav, Dinesh Nagpal",2000000.0,Seed


Get Outliers and Remove them

In [1618]:
def get_outlier(data, column):
    """
    Function that identifies outliers from the dataframe

    Args:
        data : pandas dataframe
            Contains the data where the outliers are to be found
        column : str
            Usually a string with the name of the column
    
    Returns:
        None: prints number of outliers 
    """
    
    # calculate interquartile range
    q25, q75 = np.percentile(data2021['Amount'], 25), np.percentile(data2021['Amount'], 75)
    iqr = q75 - q25
    print('Percentiles: 25th = %.3f, 75th = %.3f, IQR = %.3f' % (q25, q75, iqr))
    
    # calculate the outlier cutoff
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    
    # identify outliers
    indx = np.where((data2021['Amount'] < lower) | (data2021['Amount'] > upper))
    print('Identified outliers: %d' % len(indx[0]))
    
  

In [1619]:
# Call the function remove_outlier()

get_outlier(data2021, 'Amount')

Percentiles: 25th = 1000000.000, 75th = 11000000.000, IQR = 10000000.000
Identified outliers: 203


Since outliers are 203 representing almost 10% of our data, we choose to ignore them.

#### Cleaning the *'Stage'* Column

Inspecting the unique values in the 'Stage' Column

In [1620]:
data2021['Stage'].unique() # Get unique values of Stage column

array(['Pre-series A', None, 'Series D', 'Series C', 'Seed', 'Series B',
       'Series E', 'Pre-seed', 'Series A', 'Pre-series B', 'Debt',
       'Upsparks', 'Bridge', 'Seed+', 'Series F2', 'Series A+',
       'Series G', 'Series F', 'Series H', 'Series B3', 'PE', 'Series F1',
       'Pre-series A1', 'Venture-Series Unknown', 'Early seed',
       'Series D1', 'Seies A', 'Pre-series', 'Series A2', 'Series I'],
      dtype=object)

Check how many records have the null value at the 'Stage' column

In [1621]:
data2021['Stage'].isnull().sum()  # Get the numer of missing values in Stage

418

Oops! Pretty much! Let's take a deeper look at those records.

In [1622]:
data2021[data2021['Stage'].isnull()].head() # View first 5 rows whose stage column are null

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
1,upGrad,2015,Mumbai,EdTech,UpGrad is an online higher education platform.,"Mayank Kumar, Phalgun Kompalli, Ravijot Chugh,...","Unilazer Ventures, IIFL Asset Management",120000000.0,None
5,Urban Company,2014,New Delhi,Home services,Urban Company (Formerly UrbanClap) is a home a...,"Abhiraj Singh Bhal, Raghav Chandra, Varun Khaitan",Vy Capital,188000000.0,None
6,Comofi Medtech,2018,Bangalore,HealthTech,Comofi Medtech is a healthcare robotics startup.,Gururaj KB,"CIIE.CO, KIIT-TBI",200000.0,None
8,Vitra.ai,2020,Bangalore,Tech Startup,Vitra.ai is an AI-based video translation plat...,Akash Nidhi PS,Inflexor Ventures,3500000.0,None
9,Taikee,2010,Mumbai,E-commerce,"Taikee is the ISO-certified, B2B e-commerce pl...","Nidhi Ramachandran, Sachin Chhabra",None,1000000.0,None


Alright! These companies are listed on cruchbase, the leading provider of private-company prospecting and research solutions. <br>
Let's see if we can get reliable data to fill-in the nulls

In [1623]:
# A function to update multiple rows and columns with values

def update_values_(df, row_indices, column_index, values):
    """
    Updates multiple values in a DataFrame for specific rows and a specific column.

    Args:
        df (pd.DataFrame): The DataFrame to be modified.
        row_indices (list): Indices of the rows where the values will be updated.
        column_index (int or str): Index or name of the column where the values will be updated.
        values (list): List of new values to be assigned to the specified cells.

    Returns:
        pd.DataFrame: The DataFrame with the updated values.
    """
    df_modified = df.copy()
    column_index = df.columns.get_loc(column_index) if isinstance(column_index, str) else column_index

    for i, row_index in enumerate(row_indices):
        df_modified.iloc[row_index, column_index] = values[i]

    return df_modified


In [1624]:
# Define row_indices, column_index, values to be passed to the function update_values_() to insert values

row_indices = [1,5,6,8,11,13,14,15,16,24,31,34,35,36,37,40,42,46]
column_index = 8
values = ['Venture - Series Unknown','Secondary Market', 'Pre-Seed','Seed', 'Venture - Series Unknown','Pre-Seed',\
         'Series A', 'Series D','Series E','Series A','Debt','Series A','Venture - Series Unknown','Venture - Series Unknown','Venture - Series Unknown','Venture - Series Unknown','Series A', 'Seed','Equity Crowdfunding']
data2021 = update_values_(data2021, row_indices, column_index, values)

Good! I could just get a few. 

Let's have a look at what the unique values are again.

In [1625]:
data2021['Stage'].unique()  # Get unique values of stage column again

array(['Pre-series A', 'Venture - Series Unknown', 'Series D', 'Series C',
       'Seed', 'Secondary Market', 'Pre-Seed', None, 'Series A',
       'Series E', 'Series B', 'Pre-seed', 'Debt', 'Pre-series B',
       'Upsparks', 'Bridge', 'Seed+', 'Series F2', 'Series A+',
       'Series G', 'Series F', 'Series H', 'Series B3', 'PE', 'Series F1',
       'Pre-series A1', 'Venture-Series Unknown', 'Early seed',
       'Series D1', 'Seies A', 'Pre-series', 'Series A2', 'Series I'],
      dtype=object)

Ok, Let's put these values in a standard format.

In [1626]:
# Function to replace or group the funding rounds



def update_value(value):
    """
    Update the given value based on predefined replacements specific to the 'Stage' column.

    Args:
        value (str): The value to be updated.

    Returns:
        str: The updated value.
    """
    replacements = {
        r'Pre series|Early seed|Pre-series A|Pre-series A1': 'Pre-series',
        r'Seies A|Seed+|Pre-series B|Series A2': 'Series A',
        r'PE': 'Private Equity',
        r'Debt': 'Debt Financing',
        r'Seed1': 'Seed',
        r'None': 'Venture - Series Unknown',
        r'Series A+|Series B3| Series B': 'Series B',
        r'Series F2|Series F1|Series D1|Series D|Series G|Series H|Series I|Series E|Series F': 'Series C',
    }

    for pattern, replacement in replacements.items():
        value = re.sub(pattern, replacement, str(value))

    return value


# Apply the function to the 'Stage' column of the DataFrame
data2021['Stage'] = data2021['Stage'].apply(update_value)


Great! Checking........

In [1627]:
# Check the unique values of the stage column

data2021['Stage'].unique()

array(['Pre-series', 'Venture - Series Unknown', 'Series C', 'Series B',
       'Secondary Market', 'Pre-Series B', 'Pre-seed', 'Debt Financing',
       'Upsparks', 'Bridge', 'Series B+', 'Private Equity', 'Pre-series1',
       'Venture-Series Unknown'], dtype=object)

Ouch!  'Series B+',  'Pre-Series B' and 'Pre-series1' want to be treated diferently! I see. Let's implore another method!

In [1628]:
# Replace values

data2021['Stage'].replace('Series B+', 'Series B', inplace=True)
data2021['Stage'].replace('Pre-series1', 'Pre-series', inplace=True)
data2021['Stage'].replace('Pre-Series B', 'Series A', inplace=True)
data2021['Stage'].replace(' Series B', 'Series B', inplace=True)


Nice! Let's take a look at the unique values for stage column again!

In [1629]:
# Get unique valaues of the stage column again

data2021['Stage'].unique()

array(['Pre-series', 'Venture - Series Unknown', 'Series C', 'Series B',
       'Secondary Market', 'Series A', 'Pre-seed', 'Debt Financing',
       'Upsparks', 'Bridge', 'Private Equity', 'Venture-Series Unknown'],
      dtype=object)

Perfect!

### Investor Column <br>

Inspecting the unique instances of the 'Investor Column'

In [1630]:
data2021['Investor'].value_counts()

Inflection Point Ventures                                                                         23
Venture Catalysts                                                                                 11
Tiger Global                                                                                      11
Elevation Capital                                                                                  7
We Founder Circle                                                                                  6
Titan Capital                                                                                      6
Sixth Sense Ventures                                                                               5
Kalaari Capital                                                                                    5
Sequoia Capital India                                                                              5
Blue Ashva Capital                                                                         

NO anomaly spotted here, good! Let's now separate and keep first names......nooooo....wait! <br>

..before that I just saw 'http://100x.vc/' and '2000000' as an investor. Let's check that out!

In [1631]:
# Get the row/rows that have 'http://100x.vc/' as investor

data2021[(data2021['Investor'] == 'http://100x.vc/')|(data2021['Investor'] == '2000000')]

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
59,Karnival,2015,Bangalore,SaaS startup,A SAAS platform for retailers and brands to cr...,Neeraj Tyagi,http://100x.vc/,2500000.0,Series B
883,EyeMyEye,2021,Gurugram,Eyewear,"Shop the latest Eyeglasses, Sunglasses, Power ...",Ganesh Iyer,2000000,2000000.0,Pre-series


Alright, let's dig deeper by clicking the link.......Ohh, I got it! The Investor is a company named '100x.vc' <br>

Making the change........

In [1632]:
# Slice that value and asign '100x.vc'

data2021.at[59, 'Investor'] = '100x.vc'
data2021.at[877, 'Investor'] = 'LetsVenture'

In [1633]:
# Checking..........
data2021[(data2021['Investor'] == '100x.vc')|(data2021['Investor'] == 'LetsVenture')]

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
59,Karnival,2015,Bangalore,SaaS startup,A SAAS platform for retailers and brands to cr...,Neeraj Tyagi,100x.vc,2500000.0,Series B
877,Vayana Network,2017,Pune,Financial Services,Vayana Network is an electronic network that c...,Ramaswamy Iyer,LetsVenture,30000000.0,Series C
947,Trica,2021,Mumbai,Equity Management,Powering private markets - A technology platfo...,Nimesh Kampani,LetsVenture,3000000.0,Series B


Good! What next?............ Let's now clean by retaining only the first

In [1634]:
# Splitting the names separated by ',' and keeping the first

data2021["Investor"] = data2021["Investor"].map(lambda x: str(x).split(',')[0])


In [1635]:
data2021['Investor'].value_counts()

None                                             61
Inflection Point Ventures                        28
Venture Catalysts                                23
Tiger Global                                     19
Titan Capital                                    16
Y Combinator                                     14
Sequoia Capital India                            13
9Unicorns                                        12
Chiratae Ventures                                 9
Elevation Capital                                 9
Matrix Partners India                             9
Better Capital                                    8
Sixth Sense Ventures                              8
We Founder Circle                                 8
Mumbai Angels Network                             7
InnoVen Capital                                   7
Accel                                             7
JITO Angel Network                                7
Anicut Angel Fund                                 7
Kunal Shah  

In [1636]:
# Get the investor named '2000000'ArithmeticError
data2021[data2021['Investor']== '2000000']

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
883,EyeMyEye,2021,Gurugram,Eyewear,"Shop the latest Eyeglasses, Sunglasses, Power ...",Ganesh Iyer,2000000,2000000.0,Pre-series


In [1637]:
# Let's update the value

data2021 = update_values(data2021, 883, column_values={'Investor': 'Nueva Capital'})

Done!.............let's check further for missing values

In [1638]:
data2021[data2021['Investor']== 'None']

,Company_Brand,Founded,HeadQuarter,Sector,What_it_does,Founders,Investor,Amount,Stage
9,Taikee,2010,Mumbai,E-commerce,"Taikee is the ISO-certified, B2B e-commerce pl...","Nidhi Ramachandran, Sachin Chhabra",None,1000000.0,Venture - Series Unknown
30,Jambox Games,2021,Bangalore,Gaming,A competitive game publishing platform to help...,Ravi Vyas,None,1100000.0,Series B
42,LearnVern,2016,Ahmadabad,EdTech,LearnVern.com is a training portal where anyon...,"Niral Modi, Aditi Modi",None,1000000.0,Series B
45,moEVing,2021,Gurugram,Logistics,"Delivering clean, cost-effective and efficient...",VIKASH MISHRA,None,1000000.0,Series B
46,WeWork India,2017,Bangalore,Hospitality,"Transform buildings into beautiful, collaborat...",Karan Virwani,None,26000000.0,Series B
73,Alteria Capital,2018,Mumbai,FinTech,Alteria Capital is a Venture debt firm .,Vinod Murali,None,150000000000.0,Debt Financing
76,Speciale Invest,2017,Bangalore,FinTech,Speciale Invest is an early stage investor foc...,"Vishesh Rajaram, Arjun Rao",None,10000000.0,Venture - Series Unknown
97,Kirana247,2018,New Delhi,Logistics & Supply Chain,An on-demand FMCG supply chain company leverag...,"Tarun Jiwarajka, Pankhuri Jiwarajka",None,1000000.0,Venture - Series Unknown
108,Kirana247,2018,New Delhi,Logistics & Supply Chain,An on-demand FMCG supply chain company leverag...,"Tarun Jiwarajka, Pankhuri Jiwarajka",None,1000000.0,Pre-series
116,Zetwerk,2018,Bangalore,Mechanical & Industrial Engineering,ZETWERK is a universal manufacturing network p...,"Srinath Ramakkrushnan, Amrit Acharya",None,150000000.0,Series C


Update the values from 'None' to 'Unknown'

In [1639]:
# Change 'None' to 'Unknown'
data2021.loc[data2021['Investor'] == 'None', 'Investor'] = 'Unknown'


In [1640]:
# Check 

data2021.Investor.value_counts()

Unknown                                          64
Inflection Point Ventures                        28
Venture Catalysts                                23
Tiger Global                                     19
Titan Capital                                    16
Y Combinator                                     14
Sequoia Capital India                            13
9Unicorns                                        12
Chiratae Ventures                                 9
Elevation Capital                                 9
Matrix Partners India                             9
Better Capital                                    8
Sixth Sense Ventures                              8
We Founder Circle                                 8
Mumbai Angels Network                             7
InnoVen Capital                                   7
Accel                                             7
JITO Angel Network                                7
Anicut Angel Fund                                 7
Kunal Shah  

In [1641]:
# Get the number of null values in the investor column
data2021['Investor'].isnull().sum()

0

Done!

#### Sector column

In [1642]:
data2021['Sector'].head(10)

0        AI startup
1            EdTech
2            EdTech
3    B2B E-commerce
4           FinTech
5     Home services
6        HealthTech
7        HealthTech
8      Tech Startup
9        E-commerce
Name: Sector, dtype: object

In [1643]:
data2021.columns

Index(['Company_Brand', 'Founded', 'HeadQuarter', 'Sector', 'What_it_does',
       'Founders', 'Investor', 'Amount', 'Stage'],
      dtype='object')

In [1644]:
data2021.rename(columns={'Company_Brand':'Company Name',
                         'Founded': 'Year Established',
                         'HeadQuarter': 'Location',
                         'Sector': 'Industry',
                         'What_it_does': 'Company Description',
                         'Stage': 'Round/Series'}, inplace=True)

In [1645]:
data2021.columns

Index(['Company Name', 'Year Established', 'Location', 'Industry',
       'Company Description', 'Founders', 'Investor', 'Amount',
       'Round/Series'],
      dtype='object')

In [1646]:
data2021.isnull().sum()

Company Name           0
Year Established       0
Location               0
Industry               0
Company Description    0
Founders               4
Investor               0
Amount                 0
Round/Series           0
dtype: int64

In [1647]:
# Drop the 'Founders' column and introduce another column.....year_funded

data2021.drop('Founders', axis=1, inplace=True)

data2021['Year Funded'] = 2021

In [1648]:
data2021.isnull().sum()

Company Name           0
Year Established       0
Location               0
Industry               0
Company Description    0
Investor               0
Amount                 0
Round/Series           0
Year Funded            0
dtype: int64